# IMPORT DELLE LIBRERIE

In [ ]:
import sys
import argparse
import warnings
import os
import fileinput
import glob
import shutil
import subprocess
import numpy as np
import MDAnalysis as mda
from MDAnalysis.analysis import distances
import matplotlib.pyplot as plt
%matplotlib inline

# **SIMULAZIONE**

In [ ]:
#titleFunzioni da importare dal file Gromacs_Tools, ma visto che per ora non sappiamo farlo, le definiamo in questo slot

%run /content/drive/MyDrive/RDD/Gromacs_Tools.py

In [ ]:

#lancio grompp
def g_grompp(MdpFile, TopolFile, CoordFile, TprFile, IndexFile, maxwarning):
  with open('md-sim/grompp.log', 'w') as glog:
    process = subprocess.run(['gmx' ,'grompp',
                              '-c',CoordFile,
                              '-r',CoordFile,
                              '-p',TopolFile,
                              '-f',MdpFile,
                              '-n', IndexFile,
                              '-o',TprFile,
                              '-maxwarn', maxwarning],stdout=glog,stderr=glog)
    #lancio mdrun con o senza gpu
def g_mdrun(OutputFile,xtcFile ,TprFile=None,gpu=False,cpi_file=None):
  with open('md-sim/g_mdrun.log', 'w') as glog:
    if TprFile is None:
        print("\n\n provide a Tpr file")
        return 0

    if gpu:
        process = subprocess.run(['gmx', 'mdrun','-s',TprFile,
                                    '-o',OutputFile,
                                    '-x',xtcFile,
                                    '-cpi',cpi_file,
                                    '-nb', 'gpu'],
                                    stdout=glog,
                                    stderr=glog)
    else:
        process = subprocess.run(['gmx', 'mdrun','-s',TprFile,
                                '-o',OutputFile,
                                '-cpi',cpi_file,
                                '-nb', 'cpu'],
                                stdout=glog,
                                stderr=glog)

#lancio editconf
def g_editconf(TprFile, PdbFile):
  with open('md-sim/editconf.log', 'w') as glog:
    process = subprocess.run(['gmx' ,'editconf',
                              '-f',TprFile,
                              '-o',PdbFile],stdout=glog,stderr=glog)


In [ ]:
#Creazione della cartella per il salvataggio dei risultati

!mkdir md-sim
#mkdir: md-sim: File exists

In [ ]:
#@title suMD
#Inizializzazione dei parametri

m = 0 # threshold of the angular coefficient values of the straight line that interpolates the saved points
t_max = 31 # maximum threshold of bankruptcy attempts granted in the preliminary run
counter1t = 17 # maximum threshold of failed attempts granted in the SuMD run
ct02 = 19 # threshold relating to the times in which the distance between the com is between 0 and 2 Å during the final phase of unsupervised MD
ct25 = 19 # threshold relating to the times in which the distance between the com is between 2 and 5 Å during the final phase of unsupervised MD
ct59 = 19 # threshold relating to the times in which the distance between the com is between 5 and 9 Å during the final phase of unsupervised MD

#Inizializzazione
m_pirmo = 0 # value of the angular coefficient of the line which is updated and compared with m each short MD simulation
t_step = 1 # counter that is increased by 1 and compared with t_max every time there is a failed step in the preliminary run
counter1 = 0 # counter that is incremented by 1 and compared with counter1t every time there is a failed step in the SuMD run
c_step = 0 # counters that allow us to determine when a SuMD run step is productive
k_step = 0 # counters that allow us to determine when a SuMD run step is productive
d_cm_vector = np.zeros(5) # vector in which the distances of the com taken in the 5 steps are entered within each short MD simulation during the SuMD run
d_cm_out = 0 # final distance value 𝑑𝑐𝑚 stored at the end of the short MD simulations that occur once supervision is stopped
counter02 = 0 # counter that is updated and compared with ct02 every time that 𝑑𝑐𝑚𝑜𝑢𝑡 is between 0 and 2 Å
counter25 = 0 # counter that is updated and compared with ct25 every time that 𝑑𝑐𝑚𝑜𝑢𝑡 is between 2 and 5 Å
counter59 = 0 # counter that is updated and compared with ct59 every time that 𝑑𝑐𝑚𝑜𝑢𝑡 is between 5 and 9 Å

In [ ]:
def Linear_Fitting(PDB,XTC,n,frame_rate): #Funzione per il fitting lineare
  
  #Check dei file
  if os.stat(PDB).st_size == 0: #check PDB file
    print('Il file PDB è vuoto')
    return
  
  if os.stat(XTC).st_size == 0: #check XTC file
    print('Il file XTC è vuoto')
    return

  u = mda.Universe(PDB,XTC) #universe creation
  #LIG = u.select_atoms('resname LIG')
  #print(LIG)
  protein = u.select_atoms('resname SER or resname VAL or resname TYR or resname ILE or resname THR or resname GLU or resname LEU or resname ALA or resname GLY or resname ASN or resname CYS or resname TRP or resname GLN or resname PHE or resname ASH or resname PRO or resname CYX or resname HID or resname ARG or resname MET or resname LYS or resname HIE or resname ASP' )
  LIG = u.select_atoms('resname LIG')


  print(u.trajectory.n_frames)

  step = frame_rate/(int(n)-1) #calculate how many steps of the trajectory will be analyzed (used as integer)
  #print(step) #debug
  d = np.empty(len(u.trajectory[:frame_rate:int(step)]), dtype=float) #distance vector
  #print(len(d))

  p = 0 #aux variable for vector indexing
  print(len(u.trajectory)) ##PROBLEMA E' QUI 
  
  for ts in u.trajectory[:frame_rate:int(step)]: #trajectory analysis cycle
    d[p] = float(distances.distance_array(np.array(protein.center_of_mass()),np.array(LIG.center_of_mass())))
    p += 1

  print(p)
  print(ts)  
  print(d)
  print(len(d))


  x = np.linspace(0,600,int(n), dtype=float) #x-axis time reference
  m1 = np.polyfit(x,d.astype(np.float32),1) #linear fitting of distance vector

  print('\nVorresti visualizzare anche la curva non fittata? Se si premi 1, altrimenti premi un tasto qualsiasi')
  ch = input()
  if ch == str(1):
    plt.plot(x,d.astype(np.float32), label = "RAW Distance") 
    plt.xlabel('Time(ps)')
    plt.ylabel('Distance (Angstrom)')
    plt.show

  plt.plot(x, m1[0]*x + m1[1], marker='o', label = "Linear Fitting") #plotting
  plt.xlabel('Time(ps)')
  plt.ylabel('Distance (Angstrom)')
  plt.show 
  plt.legend() 
  return(m1)


def Preliminary_run(n): #Funzione per il preliminary run (a partire da NPT)
  #Simulazione MD (output: .gro e .trr)
  g_grompp('mdp/05-md.mdp', 'PDB/3RFM_LIG/topol/topol.top', 'PDB/3RFM_LIG/03-npt/npt.gro', 'risultati/preliminary.tpr', 'PDB/3RFM_LIG/index.ndx','2')
  g_mdrun('risultati/preliminary.trr','risultati/preliminary.xtc',TprFile='risultati/preliminary.tpr',gpu=False,cpi_file='')
  #Il cpi_file si utilizza per far riprendere la simulazione mdrun da dove aveva finito (nelle condizioni in cui m<0).
  
  g_editconf('risultati/preliminary.tpr', 'risultati/npt_nuovo.pdb') # per avere il file pdb

  m_primo = Linear_Fitting('risultati/npt_nuovo.pdb','risultati/preliminary.xtc',n,frame_rate=500)
  return m_primo

In [ ]:
!mkdir risultati #Cartella per raccolgiere i risultati
!mkdir md-sim #Cartella per raccolgiere i log

In [ ]:
#@title MODIFICA DEL FILE INDICE PER LA CREAZIONE DEL GRUPPO LIG+PROTEIN
!gmx make_ndx -f drive/MyDrive/RDD/PDB/3RFM_LIG/04-md/md.gro -n drive/MyDrive/RDD/PDB/3RFM_LIG/index.ndx -o drive/MyDrive/RDD/PDB/3RFM_LIG/index_PROLIG_new.ndx

In [ ]:
#@title CREAZIONE DEL FILE .gro
!gmx editconf -f drive/MyDrive/RDD/PDB/3RFM_LIG/04-md/md.gro -n drive/MyDrive/RDD/PDB/3RFM_LIG/index_PROLIG_new.ndx -o drive/MyDrive/RDD/PDB/3RFM_LIG/PROLIG_clear.gro

In [ ]:
#@title CREAZIONE DELL' .xtc
!gmx trjconv -s drive/MyDrive/RDD/PDB/3RFM_LIG/PROLIG_clear.gro -n drive/MyDrive/RDD/PDB/3RFM_LIG/index_PROLIG_new.ndx -f drive/MyDrive/RDD/PDB/3RFM_LIG/04-md/md.trr -o risultati/md.xtc -dump 0

In [ ]:
!gmx check -f risultati/md.xtc #check frame rate


In [ ]:
#@title PROVA
g_editconf('drive/MyDrive/RDD/PDB/3RFM_LIG/PROLIG_clear.gro', 'risultati/md.pdb') # per avere il file pdb
m_primo = Linear_Fitting('risultati/md.pdb','risultati/md.xtc',n=5,frame_rate=600)

# **Codice per il run completo della suMD**

In [ ]:
#Primo passo da NPT
#cpi no all'inizio

genius = 0
while genius == 0:
  print('\nInserisci numero step (compreso tra 3 e 302)') #step check 
  n = 5
  if int(n)<3:
    print('\nCon meno di tre step il metodo dei minimi quadrati non è applicabile, inserisci un numero più alto')
  elif int(n)>302:
    print('\nHai inserito un numero di step maggiore del numero di frame della simulazione\nOgni step deve durare almeno un frame, inserisci un numero più basso')
  else: 
    genius=1

t_step = 0
#m_primo = Preliminary_run(n) #simulazione preliminare MD (se m_primo < 0 non si entra nel preliminary run)

g_editconf('PDB/3RFM_LIG/04-md/md.tpr', 'risultati/md.pdb') # per avere il file pdb

m_primo = Linear_Fitting('risultati/md.pdb','PDB/3RFM_LIG/04-md/md.xtc',n,frame_rate=500)

"""
#PRELIMINARY RUN
while t_step < t_max and m_primo > 0:
  t_step += 1
  #Riassegnare velocità mantenendo la geometria
  m_primo = Preliminary_run(n)

  if m_primo < 0: #Uscita dal preliminary run
    break

  if t_step > t_max:
    t_step = 0
    #Riassegnare velocità e geometria

  if t_step == 1:
    print('OK')
    break


#suMD"""